# Modelling patients yet to arrive

Here we show the use of a time-varying Poisson distribution to predict a number of patients yet to arrive with a time window (say 8 hours) of the time of prediction, by specialty. We also show the use of a function that will take ED performance targets into account when predicting the number admitted by the end of the time window

[WORK IN PROGRESS] Please check back soon